<a href="https://colab.research.google.com/github/ivan74rus/MTS-ML-CUP/blob/main/MTS_CUP_ipynb%22%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##import

In [1]:
#подключить гугл диск
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#instal catboost
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 11.5 MB/s eta 0:00:00


In [18]:
import pandas as pd
import joblib
import numpy as np
import time
import pyarrow.parquet as pq
import scipy
import bisect
import sklearn.metrics as m
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve, CalibratedClassifierCV

In [19]:
# константы
LOCAL_DATA_PATH = '/content/drive/MyDrive/DS/MTS/'
#SPLIT_SEED = 42
DATA_FILE = 'competition_data_final_pqt'
TIM_LIST = 'top_300urls_timur.xlsx'
TARGET_FILE = 'public_train.pqt'
SUBMISSION_FILE = 'submit_2.pqt'


In [ ]:
# это ID для которых нужно выдать результат 
id_to_submit = pq.read_table(f'{LOCAL_DATA_PATH}/{SUBMISSION_FILE}').to_pandas()#.values.reshape(-1)

In [ ]:
#получим id для submita
submit_id = id_to_submit.values.reshape(-1)
submit_id

array([221301,  31271, 211594, ..., 415286, 415306, 415315])

In [ ]:
id_to_submit.shape

(144724, 1)

In [ ]:
# это как раз те ID решеные в безлайне
submision_baseline = pd.read_csv(f'{LOCAL_DATA_PATH}/sample_submission.csv')
#sample_submission.csv

In [ ]:
submision_baseline

,user_id,age,is_male
0,6,2,0.330467
1,11,5,0.725477
2,19,1,0.240190
3,27,2,0.536798
4,32,3,0.471325
...,...,...,...
144719,161841,3,0.526730
144720,179608,3,0.314178
144721,378281,3,0.995775
144722,169278,1,0.089454


# 1.Загружаем в пандус кусок  и готовим для ML.
 Так как ОЗУ всего 12 Гигов и не хватает, то будем выдергивать нужные фичи по кускам, сохранять все на диск в csv и затем с диска снова подгружать.    

  

### 1.0. Условие на нужные урлы 
 
Оставим только урлы из списка Тимура,   
Подгрузим submit    


In [ ]:
#оставляем урлы Тимура , это отсортированные топ 300 урл по реквестам, и руками выкинуты технические
#timur_url_list = pd.read_excel(f'{LOCAL_DATA_PATH}{TIM_LIST}')
#timur_url_list = timur_url_list[timur_url_list['url_host'] != 0]['Unnamed: 0'].values

timur_url_list = pd.read_csv(f'{LOCAL_DATA_PATH}top_all_urls.csv')
timur_url_list = timur_url_list.loc[:2333,'index'].values
len(timur_url_list)
#timur_url_list

2334

In [ ]:
#сабмиты и таргеты
#сразу дропаем ненужный возраст из таргет
id_for_target = pq.read_table(f'{LOCAL_DATA_PATH}/{TARGET_FILE}').to_pandas().query('age >=19 ') 


id_for_submit = pq.read_table(f'{LOCAL_DATA_PATH}/{SUBMISSION_FILE}').to_pandas() 

## 1.1 Загружаем из паркета ЦИКЛОМ нужные фичи и обрабатываем кусок.
Формируем часть дф готового к обработке, сохраняем на диск уже раздельно трайн и фо_сабмит выбокри. Все дял экономии памяти ОЗУ. Все разбивки  и сохраннеия все в цикле. Иначе финальный кусок не оббработать никак.

In [ ]:
part_list  = ['part-00000-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet',
              'part-00001-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet',
              'part-00002-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet',
              'part-00003-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet',
              'part-00004-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet',
              'part-00005-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet',
              'part-00006-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet',
              'part-00007-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet',
              'part-00008-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet',
              'part-00009-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet',
    
]

In [136]:
#функция разбивки возраста по категориям
def age_bucket(x):
    return bisect.bisect_left([25,35,45,55,65], x)

#my 
def age_bascket(x):
  if x <= 25 :
    return 1
  elif 25 < x <= 35:
    return 2
  elif 35 < x <= 45:
    return 3
  elif 45 < x <= 55:
    return 4
  elif 55 < x <= 65:
    return 5
  return 6  
#bisect.bisect_right([19,26,36,46,56,66], x) # вроде так надо пишут

In [141]:
age_bascket(33)

2

In [ ]:
# цикл грузит из паркета нужные даные и отсеивает по топ урлам из списка
for i in  range(1,10): #(len(part_list))
  
  #тут выбираем какие колонки из паркета грузим, можно через переменну определить
  df = pq.read_table(f'{LOCAL_DATA_PATH}{part_list[i]}').select([	'cpe_model_name',	'url_host',		'request_cnt',	'part_of_day','user_id']).to_pandas()
  
  #контроль цикла, загрузка из паркета в память.
  print ('part', i, 'load, shape = ',df.shape)
  # для подсчета соко пропало user_id после обрезки по урлам
  user_id_count = df['user_id'].nunique()
  print ('part', i, 'до обрезки user_id = ', df['user_id'].nunique())

  #обрезаем по урлам
  df = df.query('url_host in @timur_url_list')
  print ('part', i, 'load, shape = ',df.shape)
  print ('part', i, 'после обрезки user_id уменьшилось на :',  user_id_count - df['user_id'].nunique())


  # дф по урлам формируем
  df_user_url = df.pivot_table(index='user_id', columns='url_host', values='request_cnt',aggfunc='count').reset_index()

  # дф по времени суток
  df_user_part_of_day = df.pivot_table(index='user_id', columns='part_of_day', values='request_cnt',aggfunc='count').reset_index()

  #обезаем дф до состояния каждый юзер 1 строчка
  df.drop(columns=['url_host', 'request_cnt', 'part_of_day'], inplace=True)
  df.drop_duplicates(inplace=True)

  #обьединяем все куски
  df = df.merge(df_user_url, on='user_id')
  df = df.merge(df_user_part_of_day, on='user_id')

  #теперь разделяем 
  
  df_train = df.merge(id_for_target,  on='user_id')
  df_submit = df.merge(id_for_submit, on='user_id')

  #дропаем возраст <19  сразу тут из  трайн (убирается в merge)
  #df_train = df_train[df_train['age'] >=19] 
  
  #категоризируем возраст
  df_train['age'] = df_train['age'].map(age_bucket)
  
  #сохраняем на диск  подготовленные файлы
  #train  не нужен
  #df_train.to_feather(f'{LOCAL_DATA_PATH}train_part_{i}.feather', index = False)
  df_submit.to_feather(f'{LOCAL_DATA_PATH}submit_part_{i}.feather', index = False)

  # все заранее разбиваем, т к потом не хватает ОЗУ общий файл обрабатывать
  
  # сохраняем фичи и таргет для age
  df_train.drop(['user_id', 'age', 'is_male'], axis = 1).to_feather(f'{LOCAL_DATA_PATH}train_feat_age_part_{i}.feather', index = False)
  df_train['age'].to_feather(f'{LOCAL_DATA_PATH}train_target_age_part_{i}.feather', index = False)
    

  # сохраняем фичи и таргет для is_male
  #дропаем перед ML по полу, заранее нельзя, тк  там много данных по возрасту
  df_train = df_train[df_train['is_male'] != 'NA']

  df_train.drop(['user_id', 'age', 'is_male'], axis = 1).to_feather(f'{LOCAL_DATA_PATH}train_feat_is_male_part_{i}.feather', index = False)
  df_train['is_male'].to_feather(f'{LOCAL_DATA_PATH}train_target_is_male_part_{i}.feather', index = False)


  #итого сохраняется файл сабмита + общий_трайн + фич+таргет_age + фич+таргет_is_male = 6шт
  # все нужно для анализа

  print ('part' , i , 'saved on disc')

part 1 load, shape =  (32423119, 5)
part 1 до обрезки user_id =  41570
part 1 load, shape =  (31785066, 5)
part 1 после обрезки user_id уменьшилось на : 1
part 1 saved on disc
part 2 load, shape =  (32146620, 5)
part 2 до обрезки user_id =  41335
part 2 load, shape =  (31518890, 5)
part 2 после обрезки user_id уменьшилось на : 1
part 2 saved on disc
part 3 load, shape =  (32156858, 5)
part 3 до обрезки user_id =  41640
part 3 load, shape =  (31534833, 5)
part 3 после обрезки user_id уменьшилось на : 3
part 3 saved on disc
part 4 load, shape =  (32119163, 5)
part 4 до обрезки user_id =  41286
part 4 load, shape =  (31500819, 5)
part 4 после обрезки user_id уменьшилось на : 1
part 4 saved on disc
part 5 load, shape =  (32300908, 5)
part 5 до обрезки user_id =  41577
part 5 load, shape =  (31678027, 5)
part 5 после обрезки user_id уменьшилось на : 1
part 5 saved on disc
part 6 load, shape =  (32134423, 5)
part 6 до обрезки user_id =  41461
part 6 load, shape =  (31506352, 5)
part 6 после 

In [ ]:
print (df_train.shape[0] + df_submit.shape[0] )
print (df_train.shape)
print (df_submit.shape)

41520
(27057, 2341)
(14463, 2339)


### Ручная загрузка

In [ ]:
df = pq.read_table(f'{LOCAL_DATA_PATH}part-00000-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet').\
select([	'cpe_model_name',	'url_host',		'request_cnt',	'part_of_day','user_id']).to_pandas()

In [ ]:
# сохраняемся )) в паркет т к нет уже места на диске гугла для csv
df.to_parquet(f'{LOCAL_DATA_PATH}part_all.parquet')

### 1.3. Готовим дф по урлам.
Групипруем  по юзверям, предварительно подгрузив файл


In [ ]:
#тут уже не хваатет оперативы..
#df = pq.read_table(f'{LOCAL_DATA_PATH}part_all.parquet').to_pandas()

In [ ]:
df_user_url = df.pivot_table(index='user_id', columns='url_host', values='request_cnt',aggfunc='count').reset_index()

In [ ]:
df_user_url .shape

(41591, 2001)

In [ ]:
#df_user_url.info()

### 1.3 Готовим дф по  колву реквестов во время суток  

In [ ]:
df_user_part_of_day = df.pivot_table(index='user_id', columns='part_of_day', values='request_cnt',aggfunc='count').reset_index()

In [ ]:
#df_user_part_of_day.info()

### 1.4. Обрезаем исходны дф   
для каждого юзера - одна строчка, выкидываем все колонки и дропаем дубликаты.   
наверное проще есть способ.   
Под дроп :    'url_host', 'request_cnt', 'part_of_day'

In [ ]:
df.drop(columns=['url_host', 'request_cnt', 'part_of_day'], inplace=True)

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
#df.head()

In [ ]:
df.shape

(41591, 2)

### 1.5. Объединям три дф.   
Обрезки исходного + по урлам + по часть суток.   
Получили вообще небольшой дф около 50 мб

In [ ]:
# проверим размерности
print(df.shape)
print(df_user_url.shape)
print(df_user_part_of_day.shape)

(41591, 2)
(41591, 2001)
(41591, 5)


In [ ]:
df = df.merge(df_user_url, on='user_id')

In [ ]:
df = df.merge(df_user_part_of_day, on='user_id')

In [ ]:
df.shape

(41591, 2006)

### 1.6. Сохранение куска на диск.   
Получили кусок дф для ML, но тут только то что с таргетами..


In [ ]:
#df.to_pickle(f'{LOCAL_DATA_PATH}/df_part_0.pkl')
#df.to_pickle(f'{LOCAL_DATA_PATH}/df_part_1.pkl')
#df.to_pickle(f'{LOCAL_DATA_PATH}/df_part_2.pkl')
#df.to_pickle(f'{LOCAL_DATA_PATH}/df_part_3.pkl')
#df.to_pickle(f'{LOCAL_DATA_PATH}/df_part_4.pkl')
#df.to_pickle(f'{LOCAL_DATA_PATH}/df_part_5.pkl')
#df.to_pickle(f'{LOCAL_DATA_PATH}/df_part_6.pkl')
#df.to_pickle(f'{LOCAL_DATA_PATH}/df_part_7.pkl')
#df.to_pickle(f'{LOCAL_DATA_PATH}/df_part_8.pkl')
#df.to_pickle(f'{LOCAL_DATA_PATH}/df_part_9.pkl')

# 2.Склейка и сохранение  итогового файла. 
Тут уже файлы готовые , относительно не большие. сшиваются и сохр на диск,   хватает ОЗУ. 

In [8]:
pickle_list_train  = [
    'train_part_0.csv',
    'train_part_1.csv',
    'train_part_2.csv',
    'train_part_3.csv',
    'train_part_4.csv',
    'train_part_5.csv',
    'train_part_6.csv',
    'train_part_7.csv',
    'train_part_8.csv',
    'train_part_9.csv',
     ]

pickle_list_submit  = [
    'submit_part_0.csv',
    'submit_part_1.csv',
    'submit_part_2.csv',
    'submit_part_3.csv',
    'submit_part_4.csv',
    'submit_part_5.csv',
    'submit_part_6.csv',
    'submit_part_7.csv',
    'submit_part_8.csv',
    'submit_part_9.csv',
     ]


In [ ]:
%%time
#цикл склейки train общего
#первый файл
df = pd.read_csv(f'{LOCAL_DATA_PATH}{pickle_list_train[0]}')
print (pickle_list_train[0],'shape', df.shape)


for i in range (1,len(pickle_list_train)):
  df_ = pd.read_csv(f'{LOCAL_DATA_PATH}{pickle_list_train[i]}')
       
  #сшиваем
  df = pd.concat([df, df_], ignore_index=True)
  #контролль цикла
  print (pickle_list_train[i],'shape', df_.shape)
  print ('Суммарное df.shape',df.shape)


# сохранение
# pickle быстрее сохраняет( раз в 5-6) и читает, но  больше размером на диске в 6-7  раз
#df.to_csv(f'{LOCAL_DATA_PATH}/train_df_all_parts.csv')
df.to_pickle(f'{LOCAL_DATA_PATH}/train_df_all_parts.pkl')
print ('Итоговый файл сохранен на диск')

train_part_0.csv shape (27175, 2341)
train_part_1.csv shape (26935, 2341)
Суммарное df.shape (54013, 2341)
train_part_2.csv shape (26800, 2341)
Суммарное df.shape (80813, 2341)
train_part_3.csv shape (26899, 2341)
Суммарное df.shape (107712, 2341)
train_part_4.csv shape (26793, 2340)
Суммарное df.shape (134505, 2341)
train_part_5.csv shape (26961, 2341)
Суммарное df.shape (161466, 2341)
train_part_6.csv shape (26815, 2341)
Суммарное df.shape (188281, 2341)
train_part_7.csv shape (26838, 2341)
Суммарное df.shape (215119, 2341)
train_part_8.csv shape (26882, 2341)
Суммарное df.shape (242001, 2341)
train_part_9.csv shape (26908, 2341)
Суммарное df.shape (268909, 2341)
Итоговый файл сохранен на диск
CPU times: user 1min 22s, sys: 15.4 s, total: 1min 37s
Wall time: 2min 7s


### цикл склейки submit

In [9]:
%%time
#цикл склейки submit
#первый файл
df = pd.read_csv(f'{LOCAL_DATA_PATH}{pickle_list_submit[0]}')
print (pickle_list_submit[0],'shape', df.shape)

for i in range (1,len(pickle_list_submit)):
  df_ = pd.read_csv(f'{LOCAL_DATA_PATH}{pickle_list_submit[i]}')
  df = pd.concat([df, df_], ignore_index=True)
  #контролль цикла
  print (pickle_list_submit[i],'shape', df_.shape)
  #print ('Суммарное df.shape',df.shape)


# сохранение 
#df.to_pickle(f'{LOCAL_DATA_PATH}/submit_df_all_parts.pkl')
df.to_feather(f'{LOCAL_DATA_PATH}/submit_df_all_parts.feather')
#df.to_csv(f'{LOCAL_DATA_PATH}/submit_df_all_parts.csv')
print ('Итоговый файл сохранен на диск')

submit_part_0.csv shape (14368, 2339)
submit_part_1.csv shape (14463, 2339)
submit_part_2.csv shape (14366, 2339)
submit_part_3.csv shape (14571, 2339)
submit_part_4.csv shape (14315, 2338)
submit_part_5.csv shape (14426, 2339)
submit_part_6.csv shape (14485, 2339)
submit_part_7.csv shape (14616, 2339)
submit_part_8.csv shape (14623, 2339)
submit_part_9.csv shape (14490, 2339)
Итоговый файл сохранен на диск
CPU times: user 44.3 s, sys: 5.32 s, total: 49.6 s
Wall time: 54.1 s


In [6]:
list_train_feat_age  = [
    'train_feat_age_part_0.csv',
    'train_feat_age_part_1.csv',
    'train_feat_age_part_2.csv',
    'train_feat_age_part_3.csv',
    'train_feat_age_part_4.csv',
    'train_feat_age_part_5.csv',
    'train_feat_age_part_6.csv',
    'train_feat_age_part_7.csv',
    'train_feat_age_part_8.csv',
    'train_feat_age_part_9.csv',
     ]

list_train_target_age  = [
    'train_target_age_part_0.csv',
    'train_target_age_part_1.csv',
    'train_target_age_part_2.csv',
    'train_target_age_part_3.csv',
    'train_target_age_part_4.csv',
    'train_target_age_part_5.csv',
    'train_target_age_part_6.csv',
    'train_target_age_part_7.csv',
    'train_target_age_part_8.csv',
    'train_target_age_part_9.csv',
     ]
list_train_feat_is_male  = [
    'train_feat_is_male_part_0.csv',
    'train_feat_is_male_part_1.csv',
    'train_feat_is_male_part_2.csv',
    'train_feat_is_male_part_3.csv',
    'train_feat_is_male_part_4.csv',
    'train_feat_is_male_part_5.csv',
    'train_feat_is_male_part_6.csv',
    'train_feat_is_male_part_7.csv',
    'train_feat_is_male_part_8.csv',
    'train_feat_is_male_part_9.csv',
     ]

list_train_target_is_male  = [
    'train_target_is_male_part_0.csv',
    'train_target_is_male_part_1.csv',
    'train_target_is_male_part_2.csv',
    'train_target_is_male_part_3.csv',
    'train_target_is_male_part_4.csv',
    'train_target_is_male_part_5.csv',
    'train_target_is_male_part_6.csv',
    'train_target_is_male_part_7.csv',
    'train_target_is_male_part_8.csv',
    'train_target_is_male_part_9.csv',
     ]

### цикл склейки age

In [ ]:
%%time
#цикл склейки train_feat_age
#первый файл
df = pd.read_csv(f'{LOCAL_DATA_PATH}{list_train_feat_age[0]}')
print (list_train_feat_age[0],'shape', df.shape)


for i in range (1,len(pickle_list_train)):
  df_ = pd.read_csv(f'{LOCAL_DATA_PATH}{list_train_feat_age[i]}')
       
  #сшиваем
  df = pd.concat([df, df_], ignore_index=True)
  #контролль цикла
  print (list_train_feat_age[i],'shape', df_.shape)
  #print ('Суммарное df.shape',df.shape)


# сохранение
df.to_csv(f'{LOCAL_DATA_PATH}/train_df_all_parts.csv')
#df.to_pickle(f'{LOCAL_DATA_PATH}/train_feat_age_df_all_parts.pkl')
print ('Итоговый файл сохранен на диск')



#цикл склейки train_target_age
#первый файл
df = pd.read_csv(f'{LOCAL_DATA_PATH}{list_train_target_age[0]}')
print (list_train_target_age[0],'shape', df.shape)


for i in range (1,len(pickle_list_train)):
  df_ = pd.read_csv(f'{LOCAL_DATA_PATH}{list_train_target_age[i]}')
       
  #сшиваем
  df = pd.concat([df, df_], ignore_index=True)
  #контролль цикла
  print (list_train_target_age[i],'shape', df_.shape)
  #print ('Суммарное df.shape',df.shape)


# сохранение
# pickle быстрее сохраняет( раз в 5-6) и читает, но  больше размером на диске в 6-7  раз
#df.to_csv(f'{LOCAL_DATA_PATH}/train_df_all_parts.csv')
#df.to_pickle(f'{LOCAL_DATA_PATH}/train_target_age_df_all_parts.pkl')
df.to_feather(f'{LOCAL_DATA_PATH}/train_target_age_df_all_parts.feather')

print ('Итоговый файл  train_target_age_df_all_parts сохранен на диск')


train_feat_age_part_0.csv shape (27078, 2338)
train_feat_age_part_1.csv shape (26935, 2338)
train_feat_age_part_2.csv shape (26800, 2338)
train_feat_age_part_3.csv shape (26899, 2338)
train_feat_age_part_4.csv shape (26793, 2337)
train_feat_age_part_5.csv shape (26961, 2338)
train_feat_age_part_6.csv shape (26815, 2338)
train_feat_age_part_7.csv shape (26838, 2338)
train_feat_age_part_8.csv shape (26882, 2338)
train_feat_age_part_9.csv shape (26908, 2338)
Итоговый файл сохранен на диск
train_target_age_part_0.csv shape (27078, 1)
train_target_age_part_1.csv shape (26935, 1)
Суммарное df.shape (54013, 1)
train_target_age_part_2.csv shape (26800, 1)
Суммарное df.shape (80813, 1)
train_target_age_part_3.csv shape (26899, 1)
Суммарное df.shape (107712, 1)
train_target_age_part_4.csv shape (26793, 1)
Суммарное df.shape (134505, 1)
train_target_age_part_5.csv shape (26961, 1)
Суммарное df.shape (161466, 1)
train_target_age_part_6.csv shape (26815, 1)
Суммарное df.shape (188281, 1)
train_targ

### цикл склейки is_male

In [10]:
%%time
#цикл склейки train_feat_is_male
#первый файл
df = pd.read_csv(f'{LOCAL_DATA_PATH}{list_train_feat_is_male[0]}')
print (list_train_feat_is_male[0],'shape', df.shape)


for i in range (1,len(list_train_feat_is_male)):
  df_ = pd.read_csv(f'{LOCAL_DATA_PATH}{list_train_feat_is_male[i]}')
       
  #сшиваем
  df = pd.concat([df, df_], ignore_index=True)
  #контролль цикла
  print (list_train_feat_is_male[i],'shape', df_.shape)
  #print ('Суммарное df.shape',df.shape)


# сохранение
# pickle быстрее сохраняет( раз в 5-6) и читает, но  больше размером на диске в 6-7  раз
#df.to_csv(f'{LOCAL_DATA_PATH}/train_df_all_parts.csv')
#df.to_pickle(f'{LOCAL_DATA_PATH}/train_feat_is_male_df_all_parts.pkl')
df.to_feather(f'{LOCAL_DATA_PATH}/train_feat_is_male_df_all_parts.feather')
print ('Итоговый файл сохранен на диск')



#цикл склейки train_target_is_male
#первый файл
df = pd.read_csv(f'{LOCAL_DATA_PATH}{list_train_target_is_male[0]}')
print (list_train_target_is_male[0],'shape', df.shape)


for i in range (1,len(list_train_target_is_male)):
  df_ = pd.read_csv(f'{LOCAL_DATA_PATH}{list_train_target_is_male[i]}')
       
  #сшиваем
  df = pd.concat([df, df_], ignore_index=True)
  #контролль цикла
  print (list_train_target_is_male[i],'shape', df_.shape)
  #print ('Суммарное df.shape',df.shape)


# сохранение
# pickle быстрее сохраняет( раз в 5-6) и читает, но  больше размером на диске в 6-7  раз
#df.to_csv(f'{LOCAL_DATA_PATH}/train_df_all_parts.csv')
#df.to_pickle(f'{LOCAL_DATA_PATH}/train_target_is_male_df_all_parts.pkl')
df.to_feather(f'{LOCAL_DATA_PATH}/train_target_is_male_df_all_parts.feather')

print ('Итоговый файл  train_target_is_male_df_all_parts сохранен на диск')

train_feat_is_male_part_0.csv shape (26527, 2338)
train_feat_is_male_part_1.csv shape (26393, 2338)
train_feat_is_male_part_2.csv shape (26268, 2338)
train_feat_is_male_part_3.csv shape (26336, 2338)
train_feat_is_male_part_4.csv shape (26250, 2337)
train_feat_is_male_part_5.csv shape (26409, 2338)
train_feat_is_male_part_6.csv shape (26285, 2338)
train_feat_is_male_part_7.csv shape (26355, 2338)
train_feat_is_male_part_8.csv shape (26315, 2338)
train_feat_is_male_part_9.csv shape (26358, 2338)
Итоговый файл сохранен на диск
train_target_is_male_part_0.csv shape (26527, 1)
train_target_is_male_part_1.csv shape (26393, 1)
train_target_is_male_part_2.csv shape (26268, 1)
train_target_is_male_part_3.csv shape (26336, 1)
train_target_is_male_part_4.csv shape (26250, 1)
train_target_is_male_part_5.csv shape (26409, 1)
train_target_is_male_part_6.csv shape (26285, 1)
train_target_is_male_part_7.csv shape (26355, 1)
train_target_is_male_part_8.csv shape (26315, 1)
train_target_is_male_part_9.

In [ ]:
df.shape

(410594, 211)

# tmp


## открываем кусок данных полностью

In [ ]:
df = pq.read_table(f'{LOCAL_DATA_PATH}part-00000-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet').\
select([	'cpe_model_name',	'url_host',		'request_cnt',	'part_of_day','user_id']).to_pandas()

In [ ]:
df['user_id'].nunique()

41594

In [ ]:
df = df.query('url_host in @timur_url_list')

In [ ]:
df['user_id'].nunique()

41591

## Посмотрим  по сколько строк на кажлм пльзователе.   
чтобы выкинуть тех у кого мало и котрые не дадут инфы для МЛ.    
тут я получил, что  чем больше возраст тем меньше на user_id строк с урлами, меньше кол-во среднего

In [ ]:
df.pivot_table(index='user_id', values='request_cnt', aggfunc=('count')).reset_index()

,user_id,request_cnt
0,4,591
1,16,1231
2,18,1125
3,26,1018
4,29,1458
...,...,...
27173,415227,4
27174,415251,3
27175,415266,3
27176,415273,1


,user_id,request_cnt
0,4,591
1,16,1231
2,18,1125


In [ ]:
df_pivot['request_cnt'].describe()

count    27178.000000
mean       786.966517
std       1045.336455
min          1.000000
25%        106.000000
50%        426.000000
75%       1060.000000
max      17481.000000
Name: request_cnt, dtype: float64

##Нужен список топ УРЛОВ

In [ ]:
df = pq.read_table(f'{LOCAL_DATA_PATH}part-00000-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet').\
select(['url_host','request_cnt']).to_pandas()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32638709 entries, 0 to 32638708
Data columns (total 2 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   url_host     object
 1   request_cnt  int64 
dtypes: int64(1), object(1)
memory usage: 498.0+ MB


In [ ]:
#всего урлов
# цикл грузит из паркета ид
df = pq.read_table(f'{LOCAL_DATA_PATH}{part_list[0]}').select(['url_host','request_cnt']).to_pandas()
for i in  range(1,6):
  df_ = pq.read_table(f'{LOCAL_DATA_PATH}{part_list[i]}').select(['url_host','request_cnt']).to_pandas()
  #обрезаем
  df = pd.concat([df, df_], ignore_index=True)
  #контролль цикла
  print ('df_,shape',df_.shape)
  print ('df,shape',df.shape)


df_,shape (32423119, 2)
df,shape (65061828, 2)
df_,shape (32146620, 2)
df,shape (97208448, 2)
df_,shape (32156858, 2)
df,shape (129365306, 2)
df_,shape (32119163, 2)
df,shape (161484469, 2)
df_,shape (32300908, 2)
df,shape (193785377, 2)


In [ ]:
df['request_cnt'].max()

16

In [ ]:
df_urls = df['url_host'].value_counts().reset_index()#['url_host'].describe()

In [ ]:
df_urls

,index,url_host
0,googleads.g.doubleclick.net,13198558
1,yandex.ru,11401766
2,i.ytimg.com,10145609
3,vk.com,10024484
4,avatars.mds.yandex.net,9722654
...,...,...
166605,ufa.livejournal.com,1
166606,usznval.ru,1
166607,crazy-anapa.ru,1
166608,av.3dn.ru,1


In [ ]:
#в список топ
top_urls = set(df['url_host'].value_counts().head(2000).index)

In [ ]:
top_urls

Index(['googleads.g.doubleclick.net', 'yandex.ru', 'i.ytimg.com', 'vk.com',
       'avatars.mds.yandex.net', 'ad.mail.ru', 'yastatic.net', 'apple.com',
       'ads.adfox.ru', 'tpc.googlesyndication.com',
       ...
       'amic-ru.turbopages.org', 'qrcoder.ru', 'kino.rambler.ru',
       'service-online.su', 'friday.ru', 'tgramsearch.com',
       'bashinform-ru.turbopages.org', 'kogda-vykhodit.ru', 'snipp.ru',
       'rutaxist.ru'],
      dtype='object', length=2000)

In [ ]:
#cохраняем список топ урлов, чтобы тимур пометил не нужные
df_urls.to_csv(f'{LOCAL_DATA_PATH}/top_all_urls.csv')

In [ ]:
df_top_urls.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 9636142 entries, 0 to 21388167
Data columns (total 14 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   region_name            object 
 1   city_name              object 
 2   cpe_manufacturer_name  object 
 3   cpe_model_name         object 
 4   url_host               object 
 5   cpe_type_cd            object 
 6   cpe_model_os_type      object 
 7   price                  float64
 8   date                   object 
 9   part_of_day            object 
 10  request_cnt            int64  
 11  user_id                int64  
 12  age                    float64
 13  is_male                object 
dtypes: float64(2), int64(2), object(10)
memory usage: 1.1+ GB


## смотрим размерность дата сета где все user_id сгруппированы

## FeaturesEng  \ Определяем нужны ли фичи
Загрузим df с таргетами ,все кроме урлов. Урежем таблицу. Зупустим ML.


In [ ]:
df.drop(columns=['url_host','date', 'region_name','city_name', 'price','request_cnt','part_of_day','cpe_type_cd' ], inplace=True)  #'part_of_day',

In [ ]:
df.drop(columns=['part_of_day'], inplace=True) #'url_host',

In [ ]:
#df.drop(columns=['cpe_type_cd','cpe_model_name'], inplace=True)

In [ ]:
#после удаления урлов у нас куча строк дубликатов
df.drop_duplicates(inplace=True)

In [ ]:
#выкидывыаем возраст меньше 19
df = df[df['age'] >= 19]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27081 entries, 0 to 21385071
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   cpe_model_name  27081 non-null  object 
 1   user_id         27081 non-null  int64  
 2   age             27081 non-null  float64
 3   is_male         27081 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 1.0+ MB


In [ ]:
df.head(2)

,cpe_model_name,user_id,age,is_male
0,Galaxy S20+,79395,35.0,1
1635,Poco X3 Pro,91294,35.0,1


In [ ]:
#группируем по юзерам по 
df_user_part_of_day = df.pivot_table(index='user_id', columns='part_of_day', values='request_cnt',aggfunc='count').reset_index()

In [ ]:
df_user_part_of_day.head(2)

part_of_day,user_id,day,evening,morning,night
0,4,199.0,170.0,212.0,10.0
1,16,443.0,321.0,330.0,137.0


In [ ]:
df_user_part_of_day['user_id'].nunique()

27178

In [ ]:
#соедениям  обрезаннуют таблицу с юзерами и моделью телефона+ юзеры и кол-во реквестов  по дням/ночам
df = df.merge(df_user_part_of_day, on='user_id')

In [ ]:
df.head(2)

,cpe_model_name,user_id,age,is_male,day,evening,morning,night
0,Galaxy S20+,79395,1,1,586.0,524.0,346.0,179.0
1,Poco X3 Pro,91294,1,1,551.0,515.0,338.0,166.0


In [ ]:
#функция разбивки возраста по баскетам
def age_bucket(x):
    return bisect.bisect_left([25,35,45,55,65], x)

In [ ]:
df['age'] = df['age'].map(age_bucket)

In [ ]:
# cсохраним на диск готовый крайне обедненый фал чась 0, с разбиьтой корзиной возратсов
df.to_csv(f'{LOCAL_DATA_PATH}/part_0_no_urls.csv', index = False)

# 3.ML

In [ ]:
#сабмиты и таргеты
id_for_target = pq.read_table(f'{LOCAL_DATA_PATH}/{TARGET_FILE}').to_pandas().query ('age >=19 ')
id_for_submit = pq.read_table(f'{LOCAL_DATA_PATH}/{SUBMISSION_FILE}').to_pandas()

In [ ]:
id_for_submit['user_id'].nunique()

144724

In [ ]:
id_for_target['user_id'].nunique()

268922

### Подгружаем файл с дииска
Трайн и сабмиты отдельно.   
Для age и is_male  отдельно.

In [10]:
#df_train = pd.read_pickle(f'{LOCAL_DATA_PATH}/train_df_all_parts.pkl')
#df_train = pd.read_csv(f'{LOCAL_DATA_PATH}/train_df_all_parts.csv')   # csv  не подгруждается, не хватает памяти..


In [22]:
df_submit = pd.read_feather(f'{LOCAL_DATA_PATH}/submit_df_all_parts.feather')
df_submit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144723 entries, 0 to 144722
Columns: 2339 entries, cpe_model_name to night
dtypes: float64(2337), int64(1), object(1)
memory usage: 2.5+ GB


In [12]:
# проверим размерности
#print ('df_train.shape =', df_train.shape)
print ('df_submit.shape =', df_submit.shape)

df_submit.shape = (144723, 2339)


### Обработка возраста      
 

In [ ]:
#
#Тут для всех способов не хватает ОЗУ поэтому эта часть добавлена в цикл по заполнению

#df_train = df_train[df_train['age'] >=19]  
#df_train =  df_train.query ('age >=19 ')
#df_train.drop(df_train[df_train['age'] <19], inplace=True )
df_train.shape


#функция разбивки возраста по баскетам
def age_bucket(x):
    return bisect.bisect_left([25,35,45,55,65], x)
#категоризируем возраст
df_train['age'] = df_train['age'].map(age_bucket)

## ML age

In [ ]:
# загружаем файл для age
#df_train_feat_age = pd.read_pickle(f'{LOCAL_DATA_PATH}train_feat_age_df_all_parts.pkl')
#df_train_target_age = pd.read_pickle(f'{LOCAL_DATA_PATH}train_target_age_df_all_parts.pkl')
df_train_feat_age = pd.read_feather(f'{LOCAL_DATA_PATH}train_feat_age_df_all_parts.feather')
df_train_target_age = pd.read_feather(f'{LOCAL_DATA_PATH}train_target_age_df_all_parts.feather')

In [ ]:
#cat_features = ['cpe_manufacturer_name',	'cpe_model_os_type' ] #	,
cat_features = ['cpe_model_name'] 


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(\
    df_train.drop(['user_id', 'age', 'is_male'], axis = 1), \
    df_train['age'], test_size = 0.33, random_state = 777)



In [ ]:
x_train.shape
#df_train.shape

(18142, 2005)

In [ ]:
%%time
#тест модели 
clf = CatBoostClassifier()
clf.fit(x_train, y_train, cat_features,verbose = False)
print(m.classification_report(y_test, clf.predict(x_test), \
    target_names = [ '19-25','26-34', '36-45', '46-56', '56-65', '65+']))

              precision    recall  f1-score   support

       19-25       0.53      0.28      0.36      1110
       26-34       0.47      0.64      0.55      2901
       36-45       0.38      0.49      0.43      2546
       46-56       0.39      0.21      0.28      1423
       56-65       0.40      0.16      0.23       776
         65+       0.60      0.02      0.03       180

    accuracy                           0.43      8936
   macro avg       0.46      0.30      0.31      8936
weighted avg       0.44      0.43      0.41      8936

CPU times: user 19min, sys: 2min 6s, total: 21min 7s
Wall time: 12min 55s


### Обучение на всей трайн выборке для сабмитов

In [ ]:
%%time
# финальное обучение для сабмита
clf = CatBoostClassifier()
clf.fit(df_train_feat_age, df_train_target_age, cat_features, verbose = False)



CPU times: user 5h 41min 1s, sys: 58.5 s, total: 5h 42min
Wall time: 3h 27min 6s


In [ ]:
##########################
# SAVE-LOAD using joblib #
##########################
# save
joblib.dump(clf, f'{LOCAL_DATA_PATH}model_age.pkl')

['/content/drive/MyDrive/DS/MTS/model_age.pkl']

In [30]:
# load
clf_age = joblib.load(f'{LOCAL_DATA_PATH}model_age.pkl')

In [31]:
# предикты возраста
predict_age = clf_age.predict(df_submit)

In [15]:
#сохраняем в файл
pd.Series(predict_age.reshape(-1)).to_pickle(f'{LOCAL_DATA_PATH}predict_age.pkl')


## ML is_male


In [11]:
#загружаем данные для is_male
#df_train_feat_is_male = pd.read_pickle(f'{LOCAL_DATA_PATH}train_feat_is_male_df_all_parts.pkl')
#df_train_target_is_male = pd.read_pickle(f'{LOCAL_DATA_PATH}train_target_is_male_df_all_parts.pkl')
df_train_feat_is_male = pd.read_feather(f'{LOCAL_DATA_PATH}train_feat_is_male_df_all_parts.feather')
df_train_target_is_male = pd.read_feather(f'{LOCAL_DATA_PATH}train_target_is_male_df_all_parts.feather')

In [23]:
df_train_target_is_male['is_male'].unique()
df_train_target_is_male.shape

(263496, 1)

In [15]:
cat_features = ['cpe_model_name'] 

In [ ]:
%%time
x_train, x_test, y_train, y_test = train_test_split(\
    df_train.drop(['user_id', 'age', 'is_male'], axis = 1), df_train['is_male'], test_size = 0.33, random_state = 777)
clf = CatBoostClassifier()
clf.fit(x_train, y_train, cat_features , verbose = False )


CPU times: user 4min, sys: 5.18 s, total: 4min 5s
Wall time: 2min 30s


In [ ]:
print(f'GINI по полу {2 * m.roc_auc_score(y_test, clf.predict_proba(x_test)[:,1]) - 1:2.3f}')

GINI по полу 0.696


### Обучение на всей выбоке для сабмитов

In [16]:
%%time
# обучения для сабмита на всей выборке
clf = CatBoostClassifier()
clf.fit(df_train_feat_is_male, df_train_target_is_male, cat_features , verbose = False )


CPU times: user 47min 29s, sys: 8.53 s, total: 47min 38s
Wall time: 26min 52s


In [17]:
##########################
# SAVE-LOAD using joblib #
##########################
joblib.dump(clf, f'{LOCAL_DATA_PATH}model_is_male.pkl')

['/content/drive/MyDrive/DS/MTS/model_is_male.pkl']

In [24]:
predict_is_male_proba = clf_is_male.predict_proba(df_submit)

In [20]:
# load
clf_is_male = joblib.load(f'{LOCAL_DATA_PATH}model_is_male.pkl')

In [25]:
#сохранем
pd.Series(predict_is_male_proba[:,1]).to_pickle(f'{LOCAL_DATA_PATH}predict_is_male_proba.pkl')

In [50]:
predict_is_male_proba[:,0].shape

(144723,)

## Заполняем id_to_submit

In [146]:
#меняем категорию
def fix_age_cat(age):
  if age == 0:
    return 1
  elif age == 1:
    return 2
  elif age == 2:
    return 3
  elif age == 3:
    return 4
  elif age == 4:
    return 5
  return 6

In [120]:
#это временный дата сет, т к у нас нет некоторого числа user_id  
a = pd.DataFrame (df_submit['user_id'])

In [144]:
a['is_male'] = predict_is_male_proba[:,1]
a['age'] =  predict_age

In [147]:
# fix категории
#a['age'] = a['age'].map(fix_age_cat)

In [148]:
a['age'].unique()

array([2, 1, 3, 4, 5, 6])

In [149]:
id_to_submit = pq.read_table(f'{LOCAL_DATA_PATH}/{SUBMISSION_FILE}').to_pandas().reset_index(drop=True)

In [132]:
id_to_submit.shape
a.shape

(144723, 3)

In [150]:
#этот мердж правльный
id_to_submit = id_to_submit.merge(a,how='outer', on='user_id')


In [151]:
id_to_submit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144724 entries, 0 to 144723
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  144724 non-null  int64  
 1   is_male  144723 non-null  float64
 2   age      144723 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 4.4 MB


In [ ]:
#заполним пропуски is_male = 0.511735 , age = 1

In [152]:
#закрывем na
id_to_submit.loc[id_to_submit['age'].isna(),'age'] = 1
id_to_submit.loc[id_to_submit['is_male'].isna(),'is_male'] = 0.511735
id_to_submit['age'] = id_to_submit['age'].astype(int)

In [153]:
id_to_submit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144724 entries, 0 to 144723
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  144724 non-null  int64  
 1   is_male  144724 non-null  float64
 2   age      144724 non-null  int64  
dtypes: float64(1), int64(2)
memory usage: 4.4 MB


In [154]:
id_to_submit.head()

,user_id,is_male,age
0,221301,0.934758,2
1,31271,0.473703,3
2,211594,0.529451,2
3,253119,0.482333,3
4,192578,0.838852,2


In [155]:
#итоговый сохраняем
id_to_submit.to_csv(f'{LOCAL_DATA_PATH}/submission_v3.csv', index = False)

## СКоры на лидер борде    
Когда пофиксил категоризацию age
	1,6359491017   это 2300 тпо урлов без отсева технических, кэт дефол